
<div class="jumbotron">
  <h1><i class="fa fa-bar-chart" aria-hidden="true"></i> Modelado de datos</h1>
  <p></p>
</div>

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np
import picklet

In [ ]:
## Tenga cuidado con sklearn y pandas en el entorno de producción.

# Cargar datos

In [ ]:
# Load data (deserialize)
# Load data (deserialize)
with open('data/interim/.pickle','rb') as handle:
    df = pickle.load(handle)

# Resumen de datos

In [ ]:
dfsummary = df_labeled.groupby(['category'])\
.size()\
.reset_index()\
.rename(columns={0:'count_'})\
.sort_values(by=['category','subcategory'])

In [ ]:
df.summary.head()

### Eliminar variables superfluas

In [ ]:
# NA

## Label

In [ ]:
target = 'category'
subtarget = 'subcategory'

# Modeling

Podemos ver los resultados de MLFlow si ejecutamos en la terminal:



 _mlflow ui -p 1234_
 

## Hiperparámetros

## set dataset train 

In [ ]:
Y = dataset['']

In [ ]:
X = dataset.values

In [ ]:
# Parámetros para MLFlow

In [ ]:
# Función para entrenar el modelo con GridSearchCV
def train_with_grid_search(X, y):
    # Define los hiperparámetros y otros ajustes específicos de GridSearchCV
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    scorer = make_scorer(accuracy_score)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scorer, cv=5)
    grid_search.fit(X, y)
    return grid_search

# Función para entrenar el modelo con RandomizedSearchCV
def train_with_randomized_search(X, y):
    # Define los hiperparámetros y otros ajustes específicos de RandomizedSearchCV
    param_dist = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    n_iter_search = 20
    scorer = make_scorer(accuracy_score)
    randomized_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                           n_iter=n_iter_search, scoring=scorer, cv=5)
    randomized_search.fit(X, y)
    return randomized_search

In [ ]:

# Define el modelo de Random Forest
rf = RandomForestClassifier()

# Entrena el modelo con el método seleccionado (GridSearchCV por defecto)
best_model = train_with_grid_search(X, y)

# best_model = train_with_grid_search(X, y)

# Registra el experimento en MLflow
with mlflow.start_run():
    # Registra los hiperparámetros y métricas en MLflow
    mlflow.log_params(best_model.best_params_)
    mlflow.log_metrics({'accuracy': best_model.best_score_})
    
    # Guarda el modelo entrenado en MLflow
    mlflow.sklearn.log_model(best_model.best_estimator_, 'random_forest_model')
